In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/MobCommission/commissionV2/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "commissionerv2.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
django.setup()
from os import path, environ
from datetime import datetime
from services.zayev.environment.market_simulator import MarketSimulator
from services.zayev.service_layer.forester import Forester
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from matplotlib.dates import relativedelta


2023-12-13 12:06:26.156031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:


starting_time_step = datetime(year=1995, month=1, day=1, hour=10)

db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
env_config = {
    "db_params": db_params, 
    "max_episode_steps": 200, 
    "the_current_time_step": starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 5,
    "no_of_cmmdts": 6
} 

market = MarketSimulator(env_config=env_config)
market.reset()
print(market.the_current_time_step)

1995-01-01 10:00:00+00:00


In [3]:



nw_starting_time_step = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=2500)

nw_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nw_env_config = {
    "db_params": nw_db_params, 
    "max_episode_steps": 100, 
    "the_current_time_step": nw_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": True,
    "no_of_stocks": 5,
    "no_of_cmmdts": 6
} 

nw_market = MarketSimulator(env_config=nw_env_config)
nw_market.reset()
nw_forest = Forester(market=nw_market)
print(nw_market.the_current_time_step)

1995-04-15 14:00:00+00:00


In [4]:
forest = Forester(market=market)
forest.make_forest_data()
X_train = forest.train_input
y_train = forest.train_output
y_test = forest.test_output
X_test = forest.test_input
print(forest.market.the_current_time_step)

nw_forest.make_forest_data()
nw_X_train = nw_forest.train_input
nw_y_train = nw_forest.train_output
nw_y_test = nw_forest.test_output
nw_X_test = nw_forest.test_input
print(nw_forest.market.the_current_time_step)

0 1 {1: 1.0, 2: 16.849828194594565, 3: 239.55615691270663, 0: 1029.332320832526, 4: 1864.7473987387525}
1 2 {1: 1.0, 2: 16.849828194594565, 3: 239.55615691270663, 0: 1029.332320832526, 4: 1864.7473987387525}
2 3 {1: 1.0, 2: 16.849828194594565, 3: 239.55615691270663, 0: 1029.332320832526, 4: 1864.7473987387525}
3 4 {1: 1.0, 2: 16.849828194594565, 3: 239.55615691270663, 0: 1029.332320832526, 4: 1864.7473987387525}
4 5 {1: 1.0, 2: 16.849828194594565, 3: 239.55615691270663, 0: 1029.332320832526, 4: 1864.7473987387525}
0 1 {3: 1.0, 0: 1531.648134750191, 1: 2567.846415604444, 2: 20.469357663014915, 4: 1864.7473987387525}
1 2 {3: 1.0, 0: 1531.648134750191, 1: 2567.846415604444, 2: 20.469357663014915, 4: 1864.7473987387525}
2 3 {3: 1.0, 0: 1531.648134750191, 1: 2567.846415604444, 2: 20.469357663014915, 4: 1864.7473987387525}
3 4 {3: 1.0, 0: 1531.648134750191, 1: 2567.846415604444, 2: 20.469357663014915, 4: 1864.7473987387525}
4 5 {3: 1.0, 0: 1531.648134750191, 1: 2567.846415604444, 2: 20.46935

KeyboardInterrupt: 

In [ ]:
# rf = RandomForestClassifier()
# rf.fit(X_train, y_train)

In [7]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [73]
# max_depth = [int(x) for x in np.linspace(10, 80, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5]
# min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

best_params = {
    'n_estimators': None,
    'max_features': None,
    'max_depth': None,
    'min_samples_split': None,
    'min_samples_leaf': None,
    'bootstrap': None,
    "accuracy": None
}

best_accuracy = 0

In [8]:
iters = 1
for n_ests in n_estimators:
    for max_feat in max_features:
        for max_dept in max_depth:
            for min_sample_sp in min_samples_split:
                for min_sample_lf in min_samples_leaf:
                    for bt_strp in bootstrap:
                        rf = RandomForestClassifier(
                            n_estimators=n_ests,
                            max_features=max_feat,
                            max_depth=max_dept,
                            min_samples_split=min_sample_sp,
                            min_samples_leaf=min_sample_lf,
                            bootstrap=bt_strp
                        )
                        rf.fit(X_train, y_train)
                        nw_X_test = nw_forest.train_input
                        nw_y_pred = rf.predict(nw_X_test)

                        nw_y_test = nw_forest.train_output
                        nw_y_test_first_column = [row[1] for row in nw_y_test]
                        nw_y_pred_first_column = [row[1] for row in nw_y_pred]
                        accuracy = accuracy_score(nw_y_test_first_column, nw_y_pred_first_column)
                        print(f"{iters} Accuracy:", accuracy)
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params["n_estimators"]=n_ests
                            best_params["max_features"]=max_feat
                            best_params["max_depth"]=max_dept
                            best_params["min_samples_split"]=min_sample_sp
                            best_params["min_samples_leaf"]=min_sample_lf
                            best_params["bootstrap"]=bt_strp
                            best_params["accuracy"] = accuracy
                        iters = iters + 1

1 Accuracy: 0.1935483870967742
2 Accuracy: 0.22580645161290322
3 Accuracy: 0.21505376344086022
4 Accuracy: 0.23655913978494625
5 Accuracy: 0.21505376344086022
6 Accuracy: 0.1827956989247312
7 Accuracy: 0.22580645161290322
8 Accuracy: 0.17204301075268819
9 Accuracy: 0.17204301075268819
10 Accuracy: 0.21505376344086022
11 Accuracy: 0.21505376344086022
12 Accuracy: 0.1935483870967742
13 Accuracy: 0.1935483870967742


KeyboardInterrupt: 

In [9]:
print(best_params)
# {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 73, 'min_samples_split': 5, 'min_samples_leaf': 1, 'bootstrap': False, 'accuracy': 0.5}


{'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 73, 'min_samples_split': 5, 'min_samples_leaf': 2, 'bootstrap': False, 'accuracy': 0.23655913978494625}


In [10]:

rf = RandomForestClassifier(
    n_estimators=best_params["n_estimators"],
    max_features=best_params["max_features"],
    max_depth=best_params["max_depth"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    bootstrap=best_params["bootstrap"]
)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=73, min_samples_leaf=2,
                       min_samples_split=5, n_estimators=200)

In [12]:
nwa_starting_time_step = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=500)

nwa_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwa_env_config = {
    "db_params": nwa_db_params, 
    "max_episode_steps": 1000, 
    "the_current_time_step": nwa_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 5,
    "no_of_cmmdts": 6
} 

nwa_market = MarketSimulator(env_config=nwa_env_config)
nwa_market.reset()
nwa_forest = Forester(market=nwa_market)

nwa_forest.make_forest_data()

nwa_X_train = nwa_forest.train_input
nwa_y_train = nwa_forest.train_output

nwa_y_pred = rf.predict(nwa_X_train)

nwa_y_test = nwa_forest.train_output
nwa_y_test_first_column = [row[0] for row in nwa_y_test]
nwa_y_pred_first_column = [row[0] for row in nwa_y_pred]

accuracy = accuracy_score(nwa_y_test_first_column, nwa_y_pred_first_column)
print("Accuracy:", accuracy)

Accuracy: 0.8318227593152064


In [13]:
time_data = {}
rand_acts = nwa_forest.act_random()
state, reward, done, info = nwa_forest.market.step([rand_acts])
time_data[nwa_forest.market.the_current_time_step] = state
nwa_state = state
start_time = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=2500)
ttl_tm_stps = (nwa_forest.market.the_current_time_step.replace(tzinfo=None) - start_time).total_seconds()/(3600*2.5)
print(ttl_tm_stps, nwa_forest.market.the_current_time_step)
flat_state = np.array([nwa_forest.get_flattened_states(nwa_state)])
my_pred = rf.predict(flat_state)
print(len(nwa_y_pred), my_pred)
nwa_forest.act_random(best_acts=my_pred[0])
print(my_pred[0])

bb = np.array([15,15,15,15,15])
cc = nwa_forest.act_random(bb)
print(cc)


201.0 1995-05-06 12:30:00+00:00
993 [[ 9. 11. 11. 11.  9.]]
[ 5. 15. 15. 15.  5.]
[ 0.2  0.2  0.2  0.2  0.2 -1. ]


In [15]:
nwb_starting_time_step = datetime(year=1995, month=1, day=1, hour=10) + relativedelta(hours=500)

nwb_db_params = {
    'database': environ.get("POSTGRES_DB"),
    'user': environ.get("POSTGRES_USER"),
    'password': environ.get("POSTGRES_PASSWORD"),
    'host': environ.get("DB_HOST"),
    'port': environ.get("DB_PORT"),
}
nwb_env_config = {
    "db_params": nwb_db_params, 
    "max_episode_steps": 2000, 
    "the_current_time_step": nwb_starting_time_step,
    "print_output": False,
    "is_test": False,
    "test_steps": 200,
    "n_step_stocks": 5,
    "n_step_cmmdties": 5,
    "preparing": False,
    "no_of_stocks": 5,
    "no_of_cmmdts": 6
} 

nwb_market = MarketSimulator(env_config=nwb_env_config)
market.reset()
for indx in range(1000):
    if indx < 7:
        acts = np.array([15,15,15,15,15])
        rand_acts = nwa_forest.act_random(acts)
        nwb_market.step(rand_acts)
    else:
        flat_state = np.array([nwa_forest.get_flattened_states(nwb_market.state)])
        my_pred = rf.predict(flat_state)
        acts = np.array([15,15,15,15,15])
        acts = my_pred[0]
        rand_acts = nwa_forest.act_random(acts)
        nwb_market.step(rand_acts)

print(nwb_market.wallet_state)

55753.25718548302


In [16]:

total_potfolio = nwb_market.wallet_state
for ind in range(5):
    data_index = ind+1
    total_potfolio = total_potfolio + nwb_market.stock_data[data_index]["price_snapshot"]*nwb_market.shares_data[data_index]
print(total_potfolio)
print(nwb_market.the_current_time_step - relativedelta(hours=100))

809595.6593309238
1995-05-02 06:00:00+00:00


In [ ]:
4,769,689,... 1,240,887

224,133,.. 336,610